In [5]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-234996
Azure region: eastus2
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-234996


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "projectCluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, using it")
except ComputeTargetException:
    print("Creating a new compute cluster")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes = 4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())




Found existing cluster, using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-06-04T13:18:49.359000+00:00', 'errors': None, 'creationTime': '2023-06-04T13:18:45.609817+00:00', 'modifiedTime': '2023-06-04T13:18:49.286608+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


In [27]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.001, 0.01, 0.1, 1, 10, 100, 1000),
    '--max_iter': choice(50,150,300, 500)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# # Setup environment for your training run
# sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# # Create a ScriptRunConfig Object to specify the configuration details of your training job
# src = ScriptRunConfig(source_directory = "./", compute_target=compute_target, script='train.py', environment=sklearn_env)
# Create a SKLearn Estimator to specify the configuration details of your training job
est = SKLearn(source_directory = "./", compute_target=compute_target, entry_script='train.py')

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,primary_metric_name='Accuracy',primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, hyperparameter_sampling=ps, policy=policy, max_total_runs = 20)

In [19]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

hyperdrive_run.wait_for_completion(show_output=True)


RunId: HD_2d32f698-e07d-485e-a82a-ba7370259a77
Web View: https://ml.azure.com/runs/HD_2d32f698-e07d-485e-a82a-ba7370259a77?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-234996/workspaces/quick-starts-ws-234996&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-06-04T14:26:48.844451][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space
[2023-06-04T14:26:49.2386021Z][SCHEDULER][INFO]Scheduling job, id='HD_2d32f698-e07d-485e-a82a-ba7370259a77_0' 
[2023-06-04T14:26:49.3724332Z][SCHEDULER][INFO]Scheduling job, id='HD_2d32f698-e07d-485e-a82a-ba7370259a77_1' 
[2023-06-04T14:26:49.3846117Z][SCHEDULER][INFO]Scheduling job, id='HD_2d32f698-e07d-485e-a82a-ba7370259a77_2' 
[2023-06-04T14:26:49.4492608Z][SCHEDULER][INFO]Scheduling job, id='HD_2d32f698-e07d-485e-a82a-ba7370259a77_3' 
[2023-06-04T14:26:49.5541336Z][SCHEDULER][INFO]Scheduling job, id='HD_2d32f698-e07d-485e-a82a-ba7370259a77_4' 
[2023-06-0

{'runId': 'HD_2d32f698-e07d-485e-a82a-ba7370259a77',
 'target': 'projectCluster',
 'status': 'Completed',
 'startTimeUtc': '2023-06-04T14:26:48.24443Z',
 'endTimeUtc': '2023-06-04T14:33:52.022408Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '3b85e5db-81ef-4d6c-8cf1-c1113b67849f',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': '28',
  'score': '0.9061665452779801',
  'best_child_run_id': 'HD_2d32f698-e07d-485e-a82a-ba7370259a77_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_2d32f698-e07d-485e-a82a-ba7370259a77_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClien

In [20]:
import joblib
# Get your best run and save the model from that run.

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))


best_run = hyperdrive_run.get_best_run_by_primary_metric()
print("Best run metrics :", best_run.get_metrics())
print("Best run details :", best_run.get_details())
print("Best run file names :", best_run.get_file_names())



[{'run_id': 'HD_2d32f698-e07d-485e-a82a-ba7370259a77_14', 'hyperparameters': None, 'best_primary_metric': 0.9061665452779801, 'status': 'Completed'}, {'run_id': 'HD_2d32f698-e07d-485e-a82a-ba7370259a77_13', 'hyperparameters': None, 'best_primary_metric': 0.9061665452779801, 'status': 'Completed'}, {'run_id': 'HD_2d32f698-e07d-485e-a82a-ba7370259a77_1', 'hyperparameters': None, 'best_primary_metric': 0.9061665452779801, 'status': 'Completed'}, {'run_id': 'HD_2d32f698-e07d-485e-a82a-ba7370259a77_4', 'hyperparameters': None, 'best_primary_metric': 0.9061665452779801, 'status': 'Completed'}, {'run_id': 'HD_2d32f698-e07d-485e-a82a-ba7370259a77_19', 'hyperparameters': None, 'best_primary_metric': 0.9048312697256615, 'status': 'Completed'}, {'run_id': 'HD_2d32f698-e07d-485e-a82a-ba7370259a77_15', 'hyperparameters': None, 'best_primary_metric': 0.9048312697256615, 'status': 'Completed'}, {'run_id': 'HD_2d32f698-e07d-485e-a82a-ba7370259a77_16', 'hyperparameters': None, 'best_primary_metric': 0.

In [28]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_2d32f698-e07d-485e-a82a-ba7370259a77_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [29]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"])

In [30]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [35]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=compute_target,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output = False)
automl_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_44dd3d6e-aa91-4c06-ba85-2d3d39e104f7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [40]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()

print(best_run)
print(fitted_model)

print("Best run metrics :", best_run.get_metrics())
print("Best run details :", best_run.get_details())




Package:azureml-automl-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-core, training version:1.50.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.6, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.5, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-defaults, training version:1.50.0, current version:1.49.0
Package:azureml-interpret, training version:1.50.0, current version:1.49.0
Package:azureml-mlflow, training version:1.50.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.50.0.post1, current version:1.49.0
Package:azureml-responsibleai, training version:1.50.0, current version:1.49.0
Package:azureml-telemetry, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.50.0, current 

Run(Experiment: udacity-project,
Id: AutoML_44dd3d6e-aa91-4c06-ba85-2d3d39e104f7_28,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': '

In [41]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_44dd3d6e-aa91-4c06-ba85-2d3d39e104f7_28,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
#compute_target.delete()